In [ ]:
import pymysql
from pymysql.err import OperationalError, IntegrityError
import datetime
from flask import Flask, jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

def create_connection():
    """ Create a database connection to a MySQL database """
    connection = None
    try:
        connection = pymysql.connect(
            host='localhost',      
            user='root',   
            password='200684',
            database='descobridores_db'
        )
        print("Successfully connected to the database")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    
    return connection

def create_message(connection, username, date, avatar_number, message, is_admin, game_id):
    cursor = connection.cursor()
    query = """
    INSERT INTO messages (username, date, avatar_number, message, is_admin, game_id)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data = (username, date, avatar_number, message, is_admin, game_id)
    try:
        cursor.execute(query, data)
        connection.commit()
        print("Message inserted successfully")
    except IntegrityError as e:
        print(f"The error '{e}' occurred")

def read_messages(connection):
    cursor = connection.cursor()
    query = "SELECT * FROM messages"
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
    except OperationalError as e:
        print(f"The error '{e}' occurred")


def upload_to_github(file_path, filename):
    # GitHub credentials and repository details
    token = 'your_github_token'
    repo_name = 'Descobridores/descobridores-versao-5'
    commit_message = f'Add {filename}'

    # Read the file content
    with open(file_path, 'rb') as file:
        content = file.read()

    # Initialize GitHub client
    g = Github(token)
    repo = g.get_repo(repo_name)

    # Create or update the file in the repository
    try:
        # Check if the file already exists
        contents = repo.get_contents(filename)
        repo.update_file(contents.path, commit_message, content, contents.sha)
    except:
        # If file does not exist, create it
        repo.create_file(filename, commit_message, content)

    # Remove the file from the local upload folder
    os.remove(file_path)



@app.route('/messages', methods=['GET'])
def get_messages():
    game_id = request.args.get('game')  # Get the 'game' parameter from the query string
    connection = create_connection()
    
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    
     # Construct the query based on whether 'game' parameter is provided
    if game_id:
        query = "SELECT * FROM messages WHERE game_id = %s"
        cursor.execute(query, (game_id,))
    else:
        query = "SELECT * FROM messages"
        cursor.execute(query)
    
    result = cursor.fetchall()
    connection.close()
    return jsonify(result)

@app.route('/games', methods=['GET'])
def get_games():
    connection = create_connection()
    
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    
    query = "SELECT * FROM games"
    cursor.execute(query)
    
    result = cursor.fetchall()
    connection.close()
    return jsonify(result)



@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        file_path = os.path.join("teste", file.filename)
        file.save(file_path)

        # Upload to GitHub
        upload_to_github(file_path, file.filename)

        return jsonify({'message': 'File uploaded successfully!'}), 200




if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Jul/2024 15:12:38] "GET /games HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 15:12:38] "GET /messages?game=1 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 15:12:38] "GET /messages?game=1 HTTP/1.1" 200 -


Successfully connected to the database
Successfully connected to the database
Successfully connected to the database


127.0.0.1 - - [29/Jul/2024 15:12:39] "GET /messages?game=20 HTTP/1.1" 200 -


Successfully connected to the database


...

Successfully connected to the database


127.0.0.1 - - [29/Jul/2024 15:13:06] "GET /messages?game=19 HTTP/1.1" 200 -


Successfully connected to the database


127.0.0.1 - - [29/Jul/2024 15:13:34] "GET /messages?game=1 HTTP/1.1" 200 -


Successfully connected to the database


In [6]:
import pymysql
from pymysql.err import OperationalError, IntegrityError

def create_connection():
    """ Create a database connection to a MySQL database """
    connection = None
    try:
        connection = pymysql.connect(
            host='localhost',       
            user='root',   
            password='200684',
            database='descobridores_db' 
        )
        print("Successfully connected to the database")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    
    return connection

def create_message(connection, username, date, avatar_number, message, is_admin, game_id):
    cursor = connection.cursor()
    query = """
    INSERT INTO messages (username, date, avatar_number, message, is_admin, game_id)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data = (username, date, avatar_number, message, is_admin, game_id)
    try:
        cursor.execute(query, data)
        connection.commit()
        print("Message inserted successfully")
    except IntegrityError as e:
        print(f"The error '{e}' occurred")

def read_messages(connection):
    cursor = connection.cursor()
    query = "SELECT * FROM messages"
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
    except OperationalError as e:
        print(f"The error '{e}' occurred")


def create_game(connection, game_name):
    cursor = connection.cursor()
    query = """
    INSERT INTO games (game_name)
    VALUES (%s)
    """
    data = (game_name)
    try:
        cursor.execute(query, data)
        connection.commit()
        print("Game inserted successfully")
    except IntegrityError as e:
        print(f"The error '{e}' occurred")

def main():
    connection = create_connection()
    if connection:
        # Example of inserting a message
        create_message(connection, "Maria", "2024-06-23 12:00:00", 1, "Hello, world!", False, 1)
        create_message(connection, "Murilo", "2024-06-25 11:00:00", 1, "Luctus sem. Phasellus tempor leo non velit tempus, et elementum nunc lacinia.", True, 1)
        create_message(connection, "Marcia", "2024-06-26 09:00:00", 1, "Lorem ipsum dolor sit amet. Sit sint provident eum quia animi quo nesciunt placeat. ", False, 1)
        create_message(connection, "Manoel", "2024-06-19 12:00:00", 1, "Ut quidem doloremque in inventore dolores et repellendus quas et esse quia.", False, 1)
        create_message(connection, "Marcos", "2024-06-23 12:00:00", 1, "Et ipsum consequatur a molestiae galisum et beatae molestiae.", False, 2)
        create_message(connection, "Mônica", "2024-06-23 12:00:00", 1, "Luctus sem. Phasellus tempor leo non velit tempus, et elementum nunc lacinia.", True, 2)
        create_message(connection, "Mariana", "2024-06-23 12:00:00", 1, "Hello, world!", False, 4)
        create_message(connection, "Moises", "2024-06-23 12:00:00", 1, "Et ipsum consequatur a molestiae galisum et beatae molestiae.", False, 4)
        create_message(connection, "Miriam", "2024-06-23 12:00:00", 1, "Meu nome é Míriam", False, 4)

        #create_game(connection, "APOSTE NA SOMA") 
        #create_game(connection, "CONSTRUINDO PIPAS") 
        #create_game(connection, "DESAFIO DA ESCADA") 
        #create_game(connection, "DESAFIO DA ILHA") 
        #create_game(connection, "DESAFIO DA PÁ SUJA") 
        #create_game(connection, "DESAFIO DAS BANDEIRAS") 
        #create_game(connection, "DESAFIO DAS VARETAS") 
        #create_game(connection, "DESAFIO DO DRAGÃO") 
        #create_game(connection, "FLORES NO JARDIM") 
        #create_game(connection, "GINCANA DAS ALTURAS") 
        #create_game(connection, "MILU E NALA")
        #create_game(connection, "MONSTROS NA RODA")
        #create_game(connection, "ORGANIZANDO O CURRAL")
        #create_game(connection, "PALITOS COLORIDOS")
        #create_game(connection, "QUANTOS RETÂNGULOS?")
        #create_game(connection, "SEQUÊNCIA DE TRIÂNGULOS")
        #create_game(connection, "TORRE DA MATILDA")
        #create_game(connection, "TRIÂNGULOS AMIGOS")
        #create_game(connection, "TRIÂNGULOS NO PENTÁGONO")
        #create_game(connection, "TRIÂNGULOS NUMÉRICOS")

        create_game(connection, "TESTE 1")
        create_game(connection, "TESTE2")
        
        # Example of reading messages
        read_messages(connection)
        
        # Close the connection
        connection.close()

if __name__ == "__main__":
    main()


Successfully connected to the database
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Message inserted successfully
Game inserted successfully
Game inserted successfully
(8, 'Maria', datetime.datetime(2024, 6, 23, 12, 0), 1, 'Hello, world!', 0, 1)
(9, 'Murilo', datetime.datetime(2024, 6, 25, 11, 0), 1, 'Luctus sem. Phasellus tempor leo non velit tempus, et elementum nunc lacinia.', 1, 1)
(10, 'Marcia', datetime.datetime(2024, 6, 26, 9, 0), 1, 'Lorem ipsum dolor sit amet. Sit sint provident eum quia animi quo nesciunt placeat. ', 0, 1)
(11, 'Manoel', datetime.datetime(2024, 6, 19, 12, 0), 1, 'Ut quidem doloremque in inventore dolores et repellendus quas et esse quia.', 0, 1)
(12, 'Marcos', datetime.datetime(2024, 6, 23, 12, 0), 1, 'Et ipsum consequatur a molestiae galisum et beatae molestiae.', 0,